In [102]:
from enum import Enum
from typing import NamedTuple

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
from pyaspeller import YandexSpeller

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.97
plt.rcParams['figure.subplot.bottom'] = 0.05
plt.rcParams['figure.subplot.hspace'] = 0.3
matplotlib.rc("image", cmap="gray_r")

# Prepare image
Загрузка, подготовка изображения для распознавания

In [43]:
# choose tool for show image

# Comment it if not use it
%matplotlib qt


class ShowTool(Enum):
    cv2= 'cv2'
    plt='plt'

SHOW_STYLE = ShowTool.plt

In [44]:
match SHOW_STYLE:
    case ShowTool.plt:

        def show(image: np.ndarray, y_size: int = 960):
            """
            Show image in cv2 window

            :param image: Image to show
            :param y_size: Window scaler, defaults to 960
            """
            plt.imshow(image)

    case ShowTool.cv2:

        def show(image: np.ndarray, y_size: int = 960):
            """
            Show image in cv2 window

            :param image: Image to show
            :param y_size: Window scaler, defaults to 960
            """

            dy, dx = image.shape[:2]
            size = (int(dx * y_size / dy), y_size)

            cv2.imshow('contours', cv2.resize(image, size))
            cv2.resizeWindow('contours', *size)
            cv2.waitKey()

## Load
Загружаем изображение и проверяем, правильное ли мы выбрали

In [96]:
img_real = cv2.imread(r"Data\3.jpg")
y, x, _ = img_real.shape
show(img_real)

## Align
Выравниваем (насколько возможно) изображение

In [103]:
def downscale_image(image: np.ndarray, max_size: int = 1920) -> np.ndarray:
    """
    Downscale image

    :param image: Input image
    :param max_size: Max size, defaults to 2048
    :return: Downsized image
    """

    # if max_size <= max_size:
    #     return image

    scale = max_size / max(image.shape)
    return cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)


def make_square(image: np.ndarray) -> np.ndarray:
    """
    Make square from image to rotate it without a border cropping

    :param image: Supplemented image to square
    :return: Image
    """
    y, x = image.shape[:2]
    max_side = max(y, x)

    dy = max_side - y
    dx = max_side - x

    top = dy // 2
    bottom = dy - top
    left = dx // 2
    right = dx - left
    return cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])


def align(image: np.ndarray, tol: float = 5) -> np.ndarray:
    """
    Align image

    :param image: Image to align
    :param tol: Allowable angle deviation, defaults to 5
    :return: Aligned image
    """
    image_processed = cv2.Canny(image, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    image_processed = cv2.dilate(image_processed, kernel, iterations=2)

    non_zero_coordinates = cv2.findNonZero(image_processed)
    box = cv2.minAreaRect(non_zero_coordinates)
    (x, y), angle = box[1:]

    if (-tol < angle < tol) or (90 - tol < angle < 90 + tol) or (angle < -90 + tol):
        return image

    # FIXME: choose right
    # y, x = image.shape
    rotate_M = cv2.getRotationMatrix2D((x // 2, y // 2), angle, 1)
    return cv2.warpAffine(
        image.copy(),
        rotate_M,
        (int(x), int(y)),
        cv2.INTER_CUBIC,
        cv2.BORDER_REPLICATE,
    )

In [97]:
img_gray = cv2.cvtColor(img_real, cv2.COLOR_BGR2GRAY)
img_gray = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
img_gray = downscale_image(img_gray)

# Add little borders to emphasize black spaces on borders
border_add = 10
img_gray = cv2.copyMakeBorder(
    img_gray,
    border_add,
    border_add,
    border_add,
    border_add,
    cv2.BORDER_CONSTANT,
    value=[0, 0, 0],
)
img_gray = make_square(img_gray)
show(img_gray)

# Clasterization
Находим области с текстом о объединяем эти области в кластеры - абзацы для сохранения порядка слов при распознавания и исключения разрывов текста по середине абзаца.

`Mask`

Находим границы объектов, чтобы не включать области у краев, шум и тд. Размываем изображение гауссовским ядром и применяем расширение для образования связанных блоков текста, формируя маску для каждого блока текста 

In [201]:
class Mask:
    def __init__(self, image: np.ndarray) -> None:
        splitted_image = self._split(image)
        n_rectangles, self.segmented_mask = cv2.connectedComponents(splitted_image)

        self.map = {
            i: np.uint8(self.segmented_mask == i) * i for i in range(1, n_rectangles + 1)
        }
        self.type_list = sorted(
            list(range(1, n_rectangles + 1)),
            key=lambda i: np.count_nonzero(self.map[i]),
            reverse=True,
        )

    @staticmethod
    def _split(image: np.ndarray) -> np.ndarray:
        """
        Split image to connected blocks

        :param image: Image to slit
        :return: Mask with blocks
        """
        bordered_image = cv2.Canny(image, 100, 200)
        blur_kernel = (9, 9)
        blur_image = cv2.GaussianBlur(bordered_image, blur_kernel, 2)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        dilate_image = cv2.dilate(blur_image, kernel, iterations=1)
        return dilate_image

In [202]:
mask = Mask(img_gray)
show(mask.segmented_mask)

В идеале нужно стремиться сохранить баланс между количеством несвязанных блоков и расстоянием между большими блоками. Для этого проводим кластеризацию блоков для выделения абзацев

`AttentionArea`

Обводим каждый такой блок текста (точнее его маску) прямоугольником - **фокус** и смотрим в небольшой области по краям - **область видимости** (её размеры определяются `window_border_scale`). Считаем F1 метрику на этой области. Находим блоки текста в других областях - **периферия**, сортируем в порядке возрастания числа пикселей.

Добавляем новые области из периферии к фокусу так, чтобы метрика почти не уменьшалась (допуск уменьшения - `metric_tol`)

In [207]:
class BoxProperties(NamedTuple):
    """Box parameters: x0,y0 (left, top point) x1,y1 (right, bottom point)"""

    x0: int
    y0: int
    x1: int
    y1: int


class AttentionArea:
    window_border_scale = 0.02
    """Window border thickness proportional image size"""

    def __init__(self, focus_image: np.ndarray, all_image: np.ndarray) -> None:
        """
        Create box around focus image

        :param attention_mask: Used mask to look around
        :param image: Full image
        """
        non_zero_coords = cv2.findNonZero(focus_image)
        x, y, dx, dy = cv2.boundingRect(non_zero_coords)
        self.focus_box = BoxProperties(x, y, x + dx, y + dy)

        y_full, x_full = focus_image.shape
        y_border = int(y_full * self.window_border_scale)
        x_border = int(x_full * self.window_border_scale)
        self.window_box = BoxProperties(
            max(x - x_border, 0),
            max(y - y_border, 0),
            min(x + dx + x_border, x_full),
            min(y + dy + y_border, y_full),
        )
        # self.image = all_image.copy()
        self.window_slice = (
            slice(self.window_box.y0, self.window_box.y1),
            slice(self.window_box.x0, self.window_box.x1),
        )
        self.focus_slice = (
            slice(self.focus_box.y0, self.focus_box.y1),
            slice(self.focus_box.x0, self.focus_box.x1),
        )

        self.focus = focus_image.copy()
        self.window = np.zeros_like(all_image)
        self.window[self.window_slice] = all_image[self.window_slice].copy()

    @property
    def F1_metric(self) -> float:
        """F1 metric for a clusterization quality"""
        dx, dy = self.focus_box.x1 - self.focus_box.x0, self.focus_box.y1 - self.focus_box.y0

        tp = np.count_nonzero(self.focus)
        fp = dx * dy - tp
        fn = np.count_nonzero(np.sign(self.window) - np.sign(self.focus))
        return 2 * tp / (2 * tp + fp + fn)

    def get_periphery_types(self):
        in_focus_box = self.window[self.focus_slice]
        out_focus_box = self.window
        periphery_types = []
        for i in set(np.unique(self.window)) - set(np.unique(self.focus)):
            power_in_focus_box = np.count_nonzero(
                in_focus_box[in_focus_box == i] * 1,
            )
            power_in_window_box = np.count_nonzero(
                out_focus_box[out_focus_box == i] * 1,
            )
            periphery_types.append((i, +power_in_window_box - power_in_focus_box))

        return sorted(
            periphery_types,
            key=lambda x: x[1],
            reverse=True,
        )

In [192]:
    # add_img = sum([mask.map[i] for i in area.get_periphery_types()])
    # show_img = cv2.bitwise_not(
    #     np.sign(area.focus) * 200
    #     + np.sign(mask.map[periphery_type]) * 50
    #     + np.sign(add_img) * 15
    # )
    # cv2.rectangle(
    #     show_img,
    #     (area.window_box.x0, area.window_box.y0),
    #     (area.window_box.x1, area.window_box.y1),
    #     200,
    #     2,
    # )
    # cv2.rectangle(
    #     show_img,
    #     (area.focus_box.x0, area.focus_box.y0),
    #     (area.focus_box.x1, area.focus_box.y1),
    #     200,
    #     2,
    # )

    # dy, dx = show_img.shape[:2]
    # y_size = 960
    # size = (int(dx * y_size / dy), y_size)

    # cv2.imshow('contours', cv2.resize(show_img, size))
    # cv2.resizeWindow('contours', *size)

    # cv2.waitKey()

In [208]:
types_list = mask.type_list.copy()

central_type = types_list.pop(0)
area = AttentionArea(mask.map[central_type], mask.segmented_mask)
show(area.focus)

In [168]:
show(area.window)

In [169]:
area.get_periphery_types

[(6, 210), (5, 637)]

In [142]:
show(area.window)

In [212]:
metric_tol = 0.05
types_list = mask.type_list.copy()

# TODO: create rect around each mask and compare centers

clusters = []

while len(types_list):
    central_type = types_list.pop(0)
    area = AttentionArea(mask.map[central_type], mask.segmented_mask)

    periphery_types = [i for i in area.get_periphery_types() if i[0] in types_list]
    while len(periphery_types):
        mask_type, key = periphery_types.pop(0)

        if key == 0:
            area.focus += mask.map[mask_type]
            types_list.remove(mask_type)
            print('Inside')
            continue

        new_area = AttentionArea(area.focus + mask.map[mask_type], mask.segmented_mask)

        print(round(area.F1_metric, 2), ' -> ', round(new_area.F1_metric, 2), ' : ', key)

        #
        add_img = sum([mask.map[i] for i,_ in area.get_periphery_types()])
        show_img = cv2.bitwise_not(
            np.sign(area.focus) * 200
            + np.sign(mask.map[mask_type]) * 50
            + np.sign(add_img) * 15
        )
        cv2.rectangle(
            show_img,
            (area.window_box.x0, area.window_box.y0),
            (area.window_box.x1, area.window_box.y1),
            200,
            2,
        )
        cv2.rectangle(
            show_img,
            (area.focus_box.x0, area.focus_box.y0),
            (area.focus_box.x1, area.focus_box.y1),
            200,
            2,
        )

        dy, dx = show_img.shape[:2]
        y_size = 960
        size = (int(dx * y_size / dy), y_size)

        cv2.imshow('contours', cv2.resize(show_img, size))
        cv2.resizeWindow('contours', *size)

        cv2.waitKey()


        if new_area.F1_metric > area.F1_metric * (1 - metric_tol):
            print('Bam')
            area = new_area
            periphery_types = [i for i in area.get_periphery_types() if i[0] in types_list]
            types_list.remove(mask_type)
        print()

    cluster = np.sign(area.focus) * (len(clusters) + 1)


    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    # cluster = cv2.dilate(cluster, kernel, iterations=5)
    clusters.append(cluster)


# cv2.destroyAllWindows()


plt.imshow(sum(clusters))

0.78  ->  0.78  :  210
Bam

0.78  ->  0.7  :  927

0.7  ->  0.58  :  1005

0.7  ->  0.54  :  3902

0.66  ->  0.37  :  102

0.33  ->  0.24  :  557

0.33  ->  0.28  :  619

0.33  ->  0.33  :  6416
Bam

0.33  ->  0.23  :  296

0.33  ->  0.34  :  336
Bam

0.34  ->  0.23  :  296

0.34  ->  0.3  :  687

0.34  ->  0.29  :  1011

0.34  ->  0.24  :  1542

0.34  ->  0.32  :  2731
Bam

Inside
0.33  ->  0.38  :  90
Bam

0.38  ->  0.29  :  221

0.38  ->  0.29  :  462

0.38  ->  0.31  :  1011

0.38  ->  0.3  :  1215

0.38  ->  0.35  :  2240

0.38  ->  0.36  :  2898
Bam

0.36  ->  0.32  :  247

0.36  ->  0.38  :  372
Bam

0.38  ->  0.29  :  462

0.38  ->  0.34  :  641

0.38  ->  0.31  :  1130

0.38  ->  0.31  :  1154

0.38  ->  0.36  :  1165
Bam

0.36  ->  0.33  :  641


KeyboardInterrupt: 

In [198]:
plt.imshow(sum(clusters))

# Recognize
Распознаем текст на изображении и проверяем правильности написания

In [33]:
speller = YandexSpeller()

for cluster in clusters:


In [31]:
a= AttentionArea(segmented_mask=img_gray,focus_image=clusters[1])
img_cropped = img_gray[a.focus_box.y0:a.focus_box.y1,a.focus_box.x0:a.focus_box.x1]

In [48]:
# img_cropped = cv2.bitwise_and(img_gray, img_gray, mask=clasters[1])
plt.imshow(align(img_cropped),tol=)
text = pytesseract.image_to_string(
    align2(img_cropped),
    lang='rus',
    config='--psm 3',
)
print(text)
print('*' * 50)
print(speller.spelled(text))

В Строй Орнсте мы оформлялн сертификат 150 9001 для участия в
тендере. Нас сразу привлекла стоямость в 19 тысяч рублей. И срочность:
оформлекия сертификата, так как срохи у нас «горели». По телефону:
получили грамотную консультецк по 15 и скан документа был у нас в
ээтот же день. В результате тендер мы вынграли и успешно работзем.
Спасибо СтройЮрнст, за качественную работу!

**************************************************
В Строй Орнсте мы оформляли сертификат 150 9001 для участия в
тендере. Нас сразу привлекла стоимость в 19 тысяч рублей. И срочность:
оформлекия сертификата, так как срохи у нас «горели». По телефону:
получили грамотную консультацию по 15 и скан документа был у нас в
этот же день. В результате тендер мы выиграли и успешно работаем.
Спасибо Стройюрист, за качественную работу!



In [53]:
class RecognizeResult(NamedTuple):
    text: str
    angle: int
    x: int
    y: int
    dx: int
    dy: int


res = []
for img_mask in clusters:

    img_cropped = cv2.bitwise_and(img_gray, img_gray, mask=img_mask)
    non_zero_coords = cv2.findNonZero(img_mask)
    box_cordinates = cv2.boundingRect(non_zero_coords)

    # plt.imshow(img_cropped)

    for angle in [0, -90, 90, 180]:
        text = pytesseract.image_to_string(img_cropped, lang='rus', config='--psm 3')
        text = speller.spelled(text)
        is_correct = len(text)>0
        text = text + '\n'
        if len(text) and is_correct:
            res.append(RecognizeResult(text, angle, *box_cordinates))
            break

In [54]:
angles = [i.angle for i in res]
general_angle = max(set(angles), key=angles.count)
slope = 2
metric = {
    0: lambda f: f. x + slope * f.y,
    -90: lambda f: slope * f.x - (f.y + f.dy),
    90: lambda f: -slope * (f.x + f.dx) + f.y,
    180: lambda f: -(f.x + f.dx) - slope * (f.y + f.dy),
}[general_angle]

In [55]:
print(' '.join([i.text for i in sorted(res, key=metric)]))

Еврогрупп Общество с ограниченной ответственностью.

«Торговая Компания «ЕвроГруп»
ол сковороду Борская 17 ИННЛЕТИБ259 ООЧ2960
огтниАЗеря , ОКЛОРРУНЕ ЛО «НБД-Банко г.Н- Новгороя,БИКОИ2202705
озн за ЧООО ОО 050022557 Корусченуо 1014 О.00ООДООС
"Б-тай: счгостир$2@упай пуТелефонифанс: $ (831) 253-98-60,253-37-91

 № 364 от « 28» мая 2015 г

 ©ОО«СтройЮрист»

 В Строй!Орнсте мы оформляли сертификат 15О 9001 для участия в
тендере. Нас сразу привлекла стоимость в 19 тысяч рублей. И прочность
‘оформяения сертнфиката, та как срокн у нас «горелн». ЛЮ телефону
получили грамотную консультацию по Т5 и ская документа был у ас в
‘этот же день, В результате тендер мы выиграли н успещно работаем.
Спасибо СтройЮрнся, за качественную работу?


 отзыВ

 С уважением,
Директор.

©0О «ТК Еврогру Белов Е.А





In [20]:
plt.imshow(img_gray)